In [ ]:
# Prerequisites
## !pip install datasets
## !pip install imbalanced-learn

# References
# https://huggingface.co/datasets/McAuley-Lab/Amazon-Reviews-2023

In [ ]:
#from datasets import load_dataset

In [ ]:
# dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_All_Beauty", trust_remote_code=True)
# print(dataset["full"][0])

{'rating': 5.0, 'title': 'Such a lovely scent but not overpowering.', 'text': "This spray is really nice. It smells really good, goes on really fine, and does the trick. I will say it feels like you need a lot of it though to get the texture I want. I have a lot of hair, medium thickness. I am comparing to other brands with yucky chemicals so I'm gonna stick with this. Try it!", 'images': [], 'asin': 'B00YQ6X8EO', 'parent_asin': 'B00YQ6X8EO', 'user_id': 'AGKHLEW2SOWHNMFQIJGBECAF7INQ', 'timestamp': 1588687728923, 'helpful_vote': 0, 'verified_purchase': True}


In [ ]:
# dataset["full"].to_csv("amazon_reviews.csv", index=False)

Creating CSV from Arrow format:   0%|          | 0/702 [00:00<?, ?ba/s]

226752321

In [19]:
import pandas as pd
import numpy as np

In [20]:
## Load the dataset
data = pd.read_csv("amazon_reviews.csv")
data.head()

,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase
0,5.0,Such a lovely scent but not overpowering.,This spray is really nice. It smells really go...,[],B00YQ6X8EO,B00YQ6X8EO,AGKHLEW2SOWHNMFQIJGBECAF7INQ,1588687728923,0,True
1,4.0,Works great but smells a little weird.,"This product does what I need it to do, I just...",[],B081TJ8YS3,B081TJ8YS3,AGKHLEW2SOWHNMFQIJGBECAF7INQ,1588615855070,1,True
2,5.0,Yes!,"Smells good, feels great!",[],B07PNNCSP9,B097R46CSY,AE74DYR3QUGVPZJ3P7RFWBGIX7XQ,1589665266052,2,True
3,1.0,Synthetic feeling,Felt synthetic,[],B09JS339BZ,B09JS339BZ,AFQLNQNQYFWQZPJQZS6V3NZU4QBQ,1643393630220,0,True
4,5.0,A+,Love it,[],B08BZ63GMJ,B08BZ63GMJ,AFQLNQNQYFWQZPJQZS6V3NZU4QBQ,1609322563534,0,True


In [25]:
data = data[['text', 'rating']]
data.head()

,text,rating
0,This spray is really nice. It smells really go...,5.0
1,"This product does what I need it to do, I just...",4.0
2,"Smells good, feels great!",5.0
3,Felt synthetic,1.0
4,Love it,5.0


In [27]:
data.shape

(701528, 2)

In [28]:
## Missing Values
data.isnull().sum()

text      212
rating      0
dtype: int64

In [29]:
## Checking null records
data[data['text'].isnull()]

,text,rating
9078,NaN,5.0
15159,NaN,5.0
28358,NaN,5.0
29389,NaN,4.0
34286,NaN,3.0
...,...,...
693459,NaN,4.0
700225,NaN,2.0
700357,NaN,5.0
700744,NaN,3.0


In [31]:
## Dropping null records
data = data.dropna()
data[data['text'].isnull()]

,text,rating


In [34]:
## Checking the dependent variable
data['rating'].unique()

array([5., 4., 1., 3., 2.])

In [35]:
## Checking if data is balanced
data['rating'].value_counts()

rating
5.0    420566
1.0    102067
4.0     79357
3.0     56294
2.0     43032
Name: count, dtype: int64

Imbalance dataset :(